# Withdraw Liquidity
A position $\alpha$ with $i_\alpha = i$ and $\Delta s_\alpha$ is withdrawn/destroyed. Recall that by the sign conventions, $\Delta s_\alpha < 0$.

### Constraints
Removing liquidity must leave prices $p_j^Q$ unchanged for all assets $j$. It must also leave $\frac{R_i}{S_i}$ unchanged. We furthermore require that the value of the assets withdrawn ($p_i^Q\Delta r_\alpha + \Delta q_\alpha$) is $\frac{2p_i^Q}{p_i^Q + p_\alpha}\sqrt{\frac{p_i^Q}{p_\alpha}}$ percent of the assets originally contributed to the pool.

Note that $\Delta s_\alpha$ corresponds to $-\frac{\Delta s_\alpha}{S_i}R_i$ currently. We can calculate the initial $i$ contributed by $\alpha$ as $R_i^\alpha = \sqrt{\frac{p_i^Q}{p_\alpha}} \frac{-\Delta s_\alpha}{S_i}R_i$. The value withdrawn by LP should thus be (denominated in $i$) $\frac{2p_i^Q}{p_i^Q + p_\alpha}\frac{p_i^Q}{p_\alpha} \frac{-\Delta s_\alpha}{S_i}p_\alpha R_i$.

### Requirements
$$
-\Delta s_\alpha \leq s_\alpha\\
\Delta s_\alpha < 0
$$

### Updating AMM state

$$
\begin{align}
\Delta B_i &= max\left(\frac{p_i^Q - p_\alpha}{p_i^Q + p_\alpha}\Delta s_\alpha, 0\right)\\
\Delta S_i &= \Delta s_\alpha + \Delta B_i\\
\Delta R_i &= \frac{R_i}{S_i} \Delta S_i \\
\Delta Q_i &= Q_i \frac{\Delta R_i}{R_i}\\
\Delta L &= \Delta R_i \frac{Q_i}{R_i} \frac{L}{Q}\\
\Delta T_i &= Q_i^+ \frac{R_U^+}{Q_U^+} - T_i\\
\Delta T &= \Delta T_i
\end{align}
$$


### Updating agent state

$$
\begin{align}
\Delta r_\alpha &= - \Delta R_i\\
\Delta q_\alpha &= - p_i^Q\left(\frac{2p_i^Q}{p_i^Q + p_\alpha} \frac{\Delta s_\alpha}{S_i}R_i + \Delta r_\alpha\right)
\end{align}
$$

### Updating NFT in case of partial withdraw
If $s_\alpha + \Delta s_\alpha > 0$, the LP is only partially withdrawing their liquidity.

Tracking the quantity of assets initially deposited is not required, but may be desirable for other reasons. Let us denote this quantity $r_\alpha$.
Upon *partial* liquidity withdrawal, we recalculate $r_\alpha$ as though the position is being split into two positions, and one entire position is being withdrawn.
$$
\Delta r_\alpha = r_\alpha \frac{\Delta s_\alpha}{s_\alpha}
$$

## Checking the algebra

In [1]:
from sympy.solvers import solve
from sympy import Symbol
from sympy import Max
R = Symbol('R')
Q = Symbol('Q')
S = Symbol('S')
B = Symbol('B')
pi = Symbol('pi')
p = Symbol('p')
ds = Symbol('ds')
L = Symbol('L')
Qall = Symbol('Qall')

dB = Max((pi - p)/(pi + p)*ds, 0)
dS = ds + dB
dR = R/S * dS
dQ = Q * dR/R
dr = -dR
dq = -pi*(2*pi/(pi + p)*ds/S*R + dr)
dL = dR * Q/R * L/Qall

We want
- $\frac{R_i}{S_i} = \frac{R_i + \Delta R_i}{S_i + \Delta S_i}$
- $\frac{Q_i}{R_i} = \frac{Q_i + \Delta Q_i}{R_i + \Delta R_i}$, i.e. price is unchanged
- $\frac{Q_i}{R_i}\frac{Q + L}{Q} = \frac{Q_i + \Delta Q_i}{R_i + \Delta R_i}\frac{Q + \Delta Q_i + L + \Delta L}{Q + \Delta Q_i}$, i.e. target price is unchanged
- $p_i^Q \Delta r_\alpha + \Delta q_\alpha = \frac{2 p_i^Q}{p_i^Q + p_\alpha}\frac{-\Delta s_\alpha}{S_i}p_i^Q R_i$

In [2]:
print("R/S")
lhs = R/S
rhs = (R + dR)/(S + dS)
print(lhs.equals(rhs))

print("Q/R")
lhs = Q/R
rhs = (Q + dQ)/(R + dR)
print(lhs.equals(rhs))

print("LP Returns")
lhs = pi * dr + dq
rhs = 2* pi/(pi + p) * (-ds/S) * pi * R
print(lhs.equals(rhs))

print("target price")
lhs = Q/R * (Qall + L)/Qall
rhs = (Q + dQ)/(R + dR) * (Qall + dQ + L + dL)/(Qall + dQ)
print(lhs.equals(rhs))

R/S
True
Q/R
True
LP Returns
True
target price
True
